In [32]:
# Basic setup: imports and define helper functions
import asyncio, asyncssh, sys, time

ssh_connections = {}

async def start_connection(hostname):
    if hostname not in ssh_connections:
        ssh_connections[hostname] = await asyncssh.connect(hostname)
    return ssh_connections[hostname]

async def run_server(hostname):
    conn = await start_connection(hostname)
    return (await conn.create_process('/mydata/localuser/frida/server/server'))

async def run_client(hostname, device_id):
    conn = await start_connection(hostname)
    return (await conn.create_process('/mydata/localuser/frida/apps/config-app/throughput_client/m {}'.format(device_id)))

In [35]:
# Run server for 10 seconds and then quit it, as a demo
server_task = await run_server("node0")
time.sleep(10)
server_task.terminate()

In [37]:
server_task = await run_server("node0")
client_task = await run_client("node0", "abc")
# print(await client_task.stdout.read()) # TODO this blocks indefinitely
time.sleep(15)
print(await client_task.stdout.readline())
client_task.terminate()
server_task.terminate()